In [1]:
from keras import models
from keras import layers
from keras import optimizers, losses, metrics
from keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

from konlpy.tag import Okt

In [2]:
# TAG Word
PAD = "<PADDING>" 
STA = "<START>"  
END = "<END>" 
OOV = "<OOV>"

# Tag Index
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# Data Type
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# MAXiMUM Sequnce
max_sequences = 30

# Embedding Vector Dimension
embedding_dim = 100

# LSTM Hidden Layer Dimension
lstm_hidden_dim = 128

# Filter
RE_FILTER = re.compile("[.,!?\"':;~()]")

# Chatbot Data Load
chatbot_data = pd.read_csv('smuChattinghaejoTraining.txt', names=['Q', 'A'], sep='\t')
question, answer = list(chatbot_data['Q']), list(chatbot_data['A'])

In [3]:
# Question Data Size = Answer Data Size
len(question)


21190

In [4]:
# 랜덤한 챗봇 데이터 출력 (10개)
ramdomNum = np.random.randint(0, len(question), size=10)
for i in ramdomNum:
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


Q : 평생 번호 좀 알려주세요
A : office 평생교육원교학팀 연락처

Q : 1학기 기말 언젠가요 ?
A : event now 1학기 기말고사 기간

Q : 역사콘텐츠전공 삼실 위치
A : office 역사콘텐츠전공 위치

Q : 방경란 교수님 만나고 싶어
A : professor 위치

Q : 겜과 삼실 위치좀
A : office 게임전공 위치

Q : 2학기 성적 입력 언제
A : event now 2학기 성적입력 기간

Q : 이화원 교수님 연락처가 뭐야?
A : professor 연락처

Q : 글경 어뜨케 가나용 ?
A : office 글로벌경영학과 위치

Q : 전기과 과사 연락처가 뭐야?
A : office 전기공학과 연락처

Q : 공환과 어케 가야해
A : office 공간환경학부 위치



In [7]:
def pos_tag(sentences):
    
    # KoNLPy 형태소분석기 설정
    tagger = Okt()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        # 특수기호 제거
        sentence = re.sub(RE_FILTER, "", sentence)
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        
    return sentences_pos

In [8]:
# 형태소분석 수행
question = pos_tag(question)
# Question 만 형태소 분석 실시 (Answer는 이미 정해진 타입이 있기 때문)

# 변경된 점 출력
for i in ramdomNum:
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()

Q : 평생 번호 좀 알려주세요
A : office 평생교육원교학팀 연락처

Q : 1 학기 기 말 언젠가 요
A : event now 1학기 기말고사 기간

Q : 역사 콘텐츠 전공 삼실 위치
A : office 역사콘텐츠전공 위치

Q : 방경 란 교수 님 만나고 싶어
A : professor 위치

Q : 겜 과 삼실 위치 좀
A : office 게임전공 위치

Q : 2 학기 성적 입력 언제
A : event now 2학기 성적입력 기간

Q : 이화원 교수 님 연락처 가 뭐 야
A : professor 연락처

Q : 글경 어뜨케 가나 용
A : office 글로벌경영학과 위치

Q : 전기 과 과사 연락처 가 뭐 야
A : office 전기공학과 연락처

Q : 공환 과 어케 가야해
A : office 공간환경학부 위치



In [9]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words = sorted(list(words))
words[:0] = [PAD, STA, END, OOV]



In [10]:
# 단어 개수
len(words)


549

In [11]:
# 단어 출력
words


['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '1',
 '10월',
 '10일',
 '11월',
 '11일',
 '12월',
 '1월',
 '1학기',
 '2',
 '2월',
 '2일',
 '2학기',
 '3월',
 '3일',
 '4월',
 '4일',
 '5월',
 '6월',
 '6일',
 'IOT',
 'cafeteria',
 'date',
 'event',
 'iot',
 'now',
 'office',
 'professor',
 '가',
 '가나',
 '가는',
 '가복',
 '가야해',
 '가요',
 '가족',
 '가족복지학과',
 '갈',
 '감',
 '강',
 '강성',
 '강의',
 '강의평가',
 '개강',
 '개교',
 '개교기념일',
 '개발',
 '건강',
 '게임',
 '게임전공',
 '게임학',
 '겜',
 '겨울',
 '경',
 '경과',
 '경영',
 '경영학',
 '경영학부',
 '경제',
 '경제금융학부',
 '경제학',
 '계',
 '계셔',
 '계신지',
 '계심',
 '계절',
 '계절수업',
 '고',
 '고사',
 '공',
 '공간',
 '공간환경학부',
 '공과',
 '공임',
 '공학',
 '공학교육혁신센터',
 '공환',
 '과',
 '과사',
 '관리',
 '관리팀',
 '교수',
 '교원',
 '교원인사',
 '교원인사팀',
 '교육',
 '교육학',
 '교육학과',
 '교직',
 '교직지원센터',
 '교차',
 '교학',
 '교학팀',
 '국',
 '국가',
 '국가안보학과',
 '국교',
 '국안',
 '국어',
 '국어교육',
 '국어교육과',
 '국제',
 '국제언어문화교육원',
 '국제학생지원팀',
 '권',
 '귀',
 '귀지',
 '글',
 '글경',
 '글로벌',
 '글로벌경영학과',
 '글로벌랭귀지센터',
 '금',
 '금요일',
 '금융',
 '기',
 '기간',
 '기기',
 '기념일',
 '기말',
 '기말고사',
 '기초',
 '기초교육센터',
 '기

In [12]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [13]:
np.save("word_to_index",word_to_index)
np.save("index_to_word",index_to_word)

In [14]:
# 단어 -> 인덱스
# 문장을 인덱스로 변환하여 모델 입력으로 사용
dict(list(word_to_index.items()))


{'<PADDING>': 0,
 '<START>': 1,
 '<END>': 2,
 '<OOV>': 3,
 '1': 4,
 '10월': 5,
 '10일': 6,
 '11월': 7,
 '11일': 8,
 '12월': 9,
 '1월': 10,
 '1학기': 11,
 '2': 12,
 '2월': 13,
 '2일': 14,
 '2학기': 15,
 '3월': 16,
 '3일': 17,
 '4월': 18,
 '4일': 19,
 '5월': 20,
 '6월': 21,
 '6일': 22,
 'IOT': 23,
 'cafeteria': 24,
 'date': 25,
 'event': 26,
 'iot': 27,
 'now': 28,
 'office': 29,
 'professor': 30,
 '가': 31,
 '가나': 32,
 '가는': 33,
 '가복': 34,
 '가야해': 35,
 '가요': 36,
 '가족': 37,
 '가족복지학과': 38,
 '갈': 39,
 '감': 40,
 '강': 41,
 '강성': 42,
 '강의': 43,
 '강의평가': 44,
 '개강': 45,
 '개교': 46,
 '개교기념일': 47,
 '개발': 48,
 '건강': 49,
 '게임': 50,
 '게임전공': 51,
 '게임학': 52,
 '겜': 53,
 '겨울': 54,
 '경': 55,
 '경과': 56,
 '경영': 57,
 '경영학': 58,
 '경영학부': 59,
 '경제': 60,
 '경제금융학부': 61,
 '경제학': 62,
 '계': 63,
 '계셔': 64,
 '계신지': 65,
 '계심': 66,
 '계절': 67,
 '계절수업': 68,
 '고': 69,
 '고사': 70,
 '공': 71,
 '공간': 72,
 '공간환경학부': 73,
 '공과': 74,
 '공임': 75,
 '공학': 76,
 '공학교육혁신센터': 77,
 '공환': 78,
 '과': 79,
 '과사': 80,
 '관리': 81,
 '관리팀': 82,
 '교수': 83,
 '교원': 84,
 

In [15]:
# 인덱스 -> 단어
# 모델의 예측 결과인 인덱스를 문장으로 변환시 사용
dict(list(index_to_word.items()))


{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: '1',
 5: '10월',
 6: '10일',
 7: '11월',
 8: '11일',
 9: '12월',
 10: '1월',
 11: '1학기',
 12: '2',
 13: '2월',
 14: '2일',
 15: '2학기',
 16: '3월',
 17: '3일',
 18: '4월',
 19: '4일',
 20: '5월',
 21: '6월',
 22: '6일',
 23: 'IOT',
 24: 'cafeteria',
 25: 'date',
 26: 'event',
 27: 'iot',
 28: 'now',
 29: 'office',
 30: 'professor',
 31: '가',
 32: '가나',
 33: '가는',
 34: '가복',
 35: '가야해',
 36: '가요',
 37: '가족',
 38: '가족복지학과',
 39: '갈',
 40: '감',
 41: '강',
 42: '강성',
 43: '강의',
 44: '강의평가',
 45: '개강',
 46: '개교',
 47: '개교기념일',
 48: '개발',
 49: '건강',
 50: '게임',
 51: '게임전공',
 52: '게임학',
 53: '겜',
 54: '겨울',
 55: '경',
 56: '경과',
 57: '경영',
 58: '경영학',
 59: '경영학부',
 60: '경제',
 61: '경제금융학부',
 62: '경제학',
 63: '계',
 64: '계셔',
 65: '계신지',
 66: '계심',
 67: '계절',
 68: '계절수업',
 69: '고',
 70: '고사',
 71: '공',
 72: '공간',
 73: '공간환경학부',
 74: '공과',
 75: '공임',
 76: '공학',
 77: '공학교육혁신센터',
 78: '공환',
 79: '과',
 80: '과사',
 81: '관리',
 82: '관리팀',
 83: '교수',
 84: '교원',
 

In [16]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

In [17]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)


array([348, 318, 189, 138,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [18]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

array([  1,  24, 348,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [19]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)


array([ 24, 348,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [20]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)))

# 디코더 목표를 원핫인코딩으로 변환
# 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
        one_hot_data[i, j, index] = 1

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫 번째 디코더 목표 출력
y_decoder[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [21]:
# ---------- MARK:- Model Encoder

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                dropout=0.1,
                                                recurrent_dropout=0.5,
                                                return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]



# ---------- MARK:- Model Decoder

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           dropout=0.1,
                           recurrent_dropout=0.5,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


# ---------- MARK:- Model Setting

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])    

In [22]:
# ---------- MARK:- Model Encoder

# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)



# ---------- MARK:- Predict Model Decoder

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

In [23]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        if vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence.extend([vocabulary[OOV_INDEX]])
            
        # 빈칸 추가
        sentence += ' '

    return sentence

# Model.fit

In [24]:
# 모델 교육
model.fit([x_encoder, x_decoder],
                        y_decoder,
                        epochs=50,
                        batch_size=64,
                        verbose=1)
# 나중에 챗봇으로 쓸 때 epoch랑 batch_size 수치 잘 수정하기
# 추후 빡세게 Model.fit 하고 Model 저장 후 Load해서 하기

Epoch 1/50
332/332 [==============================] - 57s 93ms/step - loss: 0.8335 - acc: 0.8907
Epoch 2/50
332/332 [==============================] - 30s 89ms/step - loss: 0.1362 - acc: 0.9643
Epoch 3/50
332/332 [==============================] - 30s 89ms/step - loss: 0.1139 - acc: 0.9693
Epoch 4/50
332/332 [==============================] - 29s 89ms/step - loss: 0.0961 - acc: 0.9715
Epoch 5/50
332/332 [==============================] - 29s 88ms/step - loss: 0.0762 - acc: 0.9769
Epoch 6/50
332/332 [==============================] - 31s 92ms/step - loss: 0.0561 - acc: 0.9826
Epoch 7/50
332/332 [==============================] - 32s 97ms/step - loss: 0.0395 - acc: 0.9877
Epoch 8/50
332/332 [==============================] - 31s 94ms/step - loss: 0.0261 - acc: 0.9925
Epoch 9/50
332/332 [==============================] - 31s 94ms/step - loss: 0.0178 - acc: 0.9945
Epoch 10/50
332/332 [==============================] - 32s 96ms/step - loss: 0.0135 - acc: 0.9958
Epoch 11/50
332/332 [========

In [25]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [26]:
# 텍스트 생성
def generate_text(input_seq):
    
    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)
    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX
    
    # 인덱스 초기화
    indexs = []
    
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)
        
        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index
        
        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    
    print(indexs)
    return sentence

In [27]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('교수님 번호가 뭐야')
input_seq


array([[ 83, 142, 206,  31, 189, 302,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [28]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


[30, 326, 2]


'professor 연락처 '

In [29]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('교수님 위치가 뭐야?')
input_seq


array([[ 83, 142, 349,  31, 189, 302,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [30]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


[30, 349, 2]


'professor 위치 '

In [31]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('컴퓨터과학과 위치가 어디야')
input_seq


array([[480,  79, 349,  31, 304, 302,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [32]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


[29, 481, 349, 2]


'office 컴퓨터과학과 위치 '

In [42]:
generate_text(make_predict_input(input()))

우리 등록기간이 언제임 ?
[26, 28, 28, 156, 2]


'event now now 등록기간 '

#### Kakao-brain의 Pororo 사용 : Text에서 PERSON을 찾을 수 있음 -> Professor 명령일 경우 PERSON을 추출하여 Professor 명령에 PERSON을 넣어줌
from pororo import Pororo
ner = Pororo(task="ner",lang='ko')


sentence = input()
pf = 'professor' in generate_text(make_predict_input(sentence))
if pf:
    for name in ner(sentence):
        if 'PERSON' in name:
            orderText = generate_text(make_predict_input(sentence)).split(" ")
            print(orderText[0] + " " +name[0]+ " "+orderText[1] )
else:
    print(generate_text(make_predict_input(sentence)))
    
#### EX) 홍길동 교수님 연락처 알려줘 -> professor 연락처 + 홍길동 -> professor 홍길동 연락처

# Model Save

In [33]:
from keras.models import load_model

model.save('chatbot_model.h5')

# Model Load

from keras.models import model_from_json
def load_model(model_filename, model_weights_filename):
    with open(model_filename, 'r', encoding='utf8') as f:
        model = model_from_json(f.read())
    model.load_weights(model_weights_filename)
    return model

encoder = load_model('encoder_model.json', 'encoder_model_weights.h5')
decoder = load_model('decoder_model.json', 'decoder_model_weights.h5')

from keras.models import load_model
model = load_model('chatbot_model.h5')

In [34]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 100)    54900       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    54900       input_2[0][0]                    
______________________________________________________________________________________________